In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
from torch.utils.data import (DataLoader,)  # Gives easier dataset managment and creates mini batches
#import torchvision  # torch package for vision related things
#import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
#import torchvision.transforms as transforms
import pandas as pd
import random
from tqdm.auto import tqdm
from transformers import DistilBertTokenizer

c:\Users\Marshall\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Run when executing on Google Colab
#from google.colab import drive
#drive.mount('/content/drive')

#!cp "/content/drive/MyDrive/Colab Notebooks/Deep Learning/DeepLProject/SpamDataset.zip" "./SpamDataset.zip"
#!unzip SpamDataset.zip

In [3]:
# Split the downloaded Youtube01-Psy.csv file into train, validation and test sets
data = pd.read_csv('SpamDataset/Youtube01-Psy.csv')
# data = pd.read_csv('TSwift_ShakeItOff_Spam.csv')
data = data.sample(frac=1).reset_index(drop=True)  # Shuffle the data
train_data = data[:int(0.7*len(data))]
val_data = data[int(0.7*len(data)):int(0.85*len(data))]
test_data = data[int(0.85*len(data)):]

# Tokenize the data using distillbert
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Turn pd dataframe into a tokenized PyTorch dataset that DataLoader can use via the SpamDataset class
class SpamDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        comment_tokens = row['CONTENT']
        label = row['CLASS']
        return comment_tokens, label
    
    def collate_fn(self, batch):
        comments = [row[0] for row in batch]
        labels = [row[1] for row in batch]
        tokens = tokenizer.batch_encode_plus(comments, truncation=True, padding='max_length', max_length=30, return_tensors='pt')
        return tokens, torch.tensor(labels)
    
train_dataset = SpamDataset(train_data)
val_dataset = SpamDataset(val_data)
test_dataset = SpamDataset(test_data)


# Load Data and collate it
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=train_dataset.collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=val_dataset.collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=test_dataset.collate_fn)

In [4]:
# Print a random sample from the train_loader to confirm correct loading and tokenization
sample = next(iter(train_loader))
print(sample)
print([samples.size() for samples in sample[0]['input_ids']])


({'input_ids': tensor([[  101,  8840,  2140,  ...,     0,     0,     0],
        [  101,  2145,  3666,  ...,     0,     0,     0],
        [  101,  4931,  4364,  ...,  2065,  2017,   102],
        ...,
        [  101,  4638,  2041,  ...,     0,     0,     0],
        [  101, 16770,  1024,  ...,  2683,  2581,   102],
        [  101,  2054,  2003,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])}, tensor([0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0]))
[torch.Size([30]), torch.Size([30]), torch.Size([30]), torch.Size([30]), torch.Size([30]), torch.Size([30]), torch.Size([30]), torch.Size([30]), torch.Size([30]), tor

In [5]:
# Recurrent neural network with LSTM (many-to-one) for sequence classification to produce a binary output
## The network includes an Embedding layer, an LSTM layer and a Linear layer
class RNN_LSTM(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, num_classes, dropout):
        super(RNN_LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x: (batch_size, max_seq_length)
        embedding = self.dropout(self.embedding(x))
        # embedding shape: (batch_size, max_seq_length, embedding_size)
        output, (h_n, c_n) = self.lstm(embedding)
        # output shape: (batch_size, max_seq_length, hidden_size)
        # h_n shape: (num_layers, batch_size, hidden_size)
        out = self.fc(h_n[-1, :, :])
        # out shape: (batch_size, num_classes)
        return out

In [6]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the hyperparameters for the LSTM model
input_size = tokenizer.vocab_size
embedding_size = 128
hidden_size = 256
num_layers = 2
num_classes = 2
dropout = 0.5
learning_rate = 0.003
num_epochs = 10


# init model
model_LSTM = RNN_LSTM(input_size, embedding_size, hidden_size, num_layers, num_classes, dropout).to(device)
criterion = nn.CrossEntropyLoss()
optimizer_LSTM = optim.Adam(model_LSTM.parameters(), lr=learning_rate)

# RNN_LSTM
print('RNN_LSTM Training')
for epoch in range(num_epochs):
    model_LSTM.train()
    loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
    for batch_idx, (data, targets) in loop:
        data = {key: value.to(device) for key, value in data.items()}
        targets = targets.to(device)

        # forward
        scores = model_LSTM(data['input_ids'])
        loss = criterion(scores, targets)

        # backward
        optimizer_LSTM.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer_LSTM.step()

        # update progress bar
        loop.set_description(f'Epoch [{epoch}/{num_epochs}]')
        loop.set_postfix(loss=loss.item())

RNN_LSTM Training


  0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Store all the predictions in model_predictions and the actual labels in actual_labels
original_comments = []
tokens = []
model_predictions = []
actual_labels = []

# Validation
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    predictions = []
    targets = []
    model.eval()

    with torch.no_grad():
        for data, target in loader:
            data = {key: value.to(device) for key, value in data.items()}
            target = target.to(device)

            scores = model(data['input_ids'])
            _, predicted = scores.max(1)
            # Store decoded original comments, model predictions and actual labels 
            original_comments.extend([tokenizer.decode(data['input_ids'][i]) for i in range(len(data['input_ids']))])
            tokens.extend(data['input_ids'])
            model_predictions.extend(predicted)
            actual_labels.extend(target)
            num_correct += (predicted == target).sum()
            num_samples += predicted.size(0)

            predictions.extend(predicted.tolist())
            targets.extend(target.tolist())

    # model.train()
    accuracy = num_correct/num_samples
    return accuracy, predictions, targets


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
train_acc, train_preds, train_targets = check_accuracy(train_loader, model_LSTM)
val_acc, val_preds, val_targets = check_accuracy(val_loader, model_LSTM)
test_acc, test_preds, test_targets = check_accuracy(test_loader, model_LSTM)

print(f'Train accuracy: {train_acc}')
print(f'Validation accuracy: {val_acc}')
print(f'Test accuracy: {test_acc}')

train_cm = confusion_matrix(train_targets, train_preds)
val_cm = confusion_matrix(val_targets, val_preds)
test_cm = confusion_matrix(test_targets, test_preds)

print(f'Train Confusion Matrix:\n{train_cm}')
print(f'Validation Confusion Matrix:\n{val_cm}')
print(f'Test Confusion Matrix:\n{test_cm}')

# Classification report
train_report = classification_report(train_targets, train_preds)
val_report = classification_report(val_targets, val_preds)
test_report = classification_report(test_targets, test_preds)

print(f'Train Classification Report:\n{train_report}')
print(f'Validation Classification Report:\n{val_report}')
print(f'Test Classification Report:\n{test_report}')

# F1 Score, Recall, Precision
# train_f1 = f1_score(train_targets, train_preds)
# val_f1 = f1_score(val_targets, val_preds)
# test_f1 = f1_score(test_targets, test_preds)

# train_recall = recall_score(train_targets, train_preds)
# val_recall = recall_score(val_targets, val_preds)
# test_recall = recall_score(test_targets, test_preds)

# train_precision = precision_score(train_targets, train_preds)
# val_precision = precision_score(val_targets, val_preds)
# test_precision = precision_score(test_targets, test_preds)

# print(f'Train F1 Score: {train_f1}')
# print(f'Validation F1 Score: {val_f1}')
# print(f'Test F1 Score: {test_f1}')

# print(f'Train Recall: {train_recall}')
#print(f'Validation Recall: {val_recall}')
# print(f'Test Recall: {test_recall}')

# print(f'Train Precision: {train_precision}')
# print(f'Validation Precision: {val_precision}')
# print(f'Test Precision: {test_precision}')

# Check accuracy on training & validation to see how good our model
# train_acc = check_accuracy(train_loader, model_LSTM)
# val_acc = check_accuracy(val_loader, model_LSTM)
# print(f'Train accuracy: {train_acc}')
# print(f'Validation accuracy: {val_acc}')
# print(f'Test accuracy: {check_accuracy(test_loader, model_LSTM)}')

# Print the original comments, model predictions and actual labels
for i in range(5):
    print(f'Comment: {original_comments[i]}')
    print(f'Tokens: {tokens[i]}')
    print(f'Prediction: {model_predictions[i]}')
    print(f'Actual: {actual_labels[i]}')
    print('')


Train accuracy: 0.9439527988433838
Validation accuracy: 0.8493150472640991
Test accuracy: 0.8082191944122314
Comment: [CLS] i still remember when this song came out i was one of those who clicked first and there were only 10 views [UNK] < br > yes im [SEP]
Tokens: tensor([  101,  1045,  2145,  3342,  2043,  2023,  2299,  2234,  2041,  1045,
         2001,  2028,  1997,  2216,  2040, 13886,  2034,  1998,  2045,  2020,
         2069,  2184,  5328,   100,  1026,  7987,  1028,  2748, 10047,   102])
Prediction: 0
Actual: 0

Comment: [CLS] okay it ’ s time to be honest. when this came out i was homeless, on drugs, and then jail. now i have a [SEP]
Tokens: tensor([  101,  3100,  2009,  1521,  1055,  2051,  2000,  2022,  7481,  1012,
         2043,  2023,  2234,  2041,  1045,  2001, 11573,  1010,  2006,  5850,
         1010,  1998,  2059,  7173,  1012,  2085,  1045,  2031,  1037,   102])
Prediction: 1
Actual: 1

Comment: [CLS] 100 % not racist even she could be saying that black people ar bett